## Feedforward Neural Language Modelling

In this section and the next we’ll introduce a simpler version of neural language models for feedforward networks, an algorithm first introduced by Bengio et al. (2003).

Neural language models have many advantages over the n-gram language models. Compared to n-gram models, neural language models can handle much longer histories, can generalize better over contexts of similar words, and are more accurate at word-prediction. On the other hand, neural net language models are much more complex, are slower and need more energy to train, and are less interpretable than n-gram models, so for some smaller tasks an n-gram language model is still the right tool.

A feedforward neural language model (LM) is a feedforward network that takes as input at time $t$ a representation of some number of previous words ($w_{t−1}$;$w_{t−2}$, etc.) and outputs a probability distribution over possible next words. Thus—like the n-gram LM—the feedforward neural LM approximates the probability of a word given the entire prior context $P(w_t|w_{1:t−1})$ by approximating based on the $N − 1$ previous words:

$$
P(w_t|w_1,\cdots,w_{t-1}) \approx P(w_t|w_{t-N+1}, \cdots, w_{t-1})
$$

In the following examples we’ll use a 4-gram example, so we’ll show a neural net to estimate the probability $P(w_t=i|w_{t−3},w_{t−2},w_{t−1})$.

Neural language models represent words in this prior context by their **embeddings**, rather than just by their word identity as used in n-gram language models. Using embeddings allows neural language models to generalize better to unseen data. For example, suppose we’ve seen this sentence in training:
<pre>
I have to make sure that the cat gets fed.
</pre>

but have never seen the words “gets fed” after the word “dog”. Our test set has the prefix “I forgot to make sure that the dog gets”. What’s the next word? An n-gram language model will predict “fed” after “that the cat gets”, but not after “that the dog gets”. But a neural LM, knowing that “cat” and “dog” have similar embeddings, will be able to generalize from the “cat” context to assign a high enough probability to “fed” even after seeing “dog”.

## Forward inference in the neural language model

Let’s walk through forward inference or decoding for neural language models. Forward inference is the task, given an input, of running a forward pass on the network to produce a probability distribution over possible outputs, in this case next words.

We first represent each of the $N$ previous words as a one-hot vector of length $|V|$, i.e., with one dimension for each word in the vocabulary. A **one-hot** vector is a vector that has one element equal to 1 — in the dimension corresponding to that word’s index in the vocabulary— while all the other elements are set to zero. Thus
in a one-hot representation for the word “toothpaste”, supposing it is $V_5$, i.e., index 5 in the vocabulary, $x_5 = 1$, and $x_i = 0 \quad \forall i \not = 5$, as shown here:

$$
\begin{matrix}
\begin{bmatrix} 0 & 0 & 0 & 0 & 1 & 0 & 0 & \cdots &  0 \end{bmatrix} \\
\begin{matrix}  1 & 2 & 3 & 4 & 5 & 6 & 7 & \cdots |V| \end{matrix}
\end{matrix}
$$

The feedforward neural language model has a moving window that can see N words into the past. We’ll let $N$ equal 3, so the 3 words $w_{t−1}$, $w_{t−2}$, and $w_{t−3}$ are each represented as a one-hot vector. We then multiply these one-hot vectors by the embedding matrix **$E$**. The embedding weight matrix **$E$** has a column for each word, each a column vector of $d$ dimensions, and hence has dimensionality $d \times |V|$. Multiplying by a one-hot vector that has only one non-zero element $x_i = 1$ simply selects out the relevant column vector for word $i$, resulting in the embedding for word $i$.

<div style="display:flex;align-items:center;justify-content:center;">
<img src="images/embedding_representation.png" />
</div>

The 3 resulting embedding vectors are concatenated to produce **$e$**, the embedding layer. This is followed by a hidden layer and an output layer whose softmax produces a probability distribution over words. For example $y_{42}$, the value of output node $42$, is the probability of the next word $w_t$ being $V_{42}$, the vocabulary word with index $42$.

Here’s the algorithm in detail for our mini example:

1. **Select three embeddings from $E$:** Given the three previous words, we look up their indices, create 3 one-hot vectors, and then multiply each by the embedding matrix $E$. Consider $w_{t−3}$. The one-hot vector for ‘for’ (index 35) is multiplied by the embedding matrix $E$, to give the first part of the first hidden layer, the **embedding layer**. Since each column of the input matrix $E$ is an layer embedding for a word, and the input is a one-hot column vector $x_i$ for word $V_i$, the embedding layer for input $w$ will be $Ex_i = e_i$, the embedding for word $i$. We now concatenate the three embeddings for the three context words to produce the embedding layer $e$.

2. **Multiply by $W$**: We multiply by $W$ (and add $b$) and pass through the ReLU (or other) activation function to get the hidden layer $h$.

3. **Multiply by $U$**: $h$ is now multiplied by $U$.

4. **Apply softmax**: After the softmax, each node $i$ in the output layer estimates the probability 
$$P(w_t = i|w_{t-1}, w_{t-2}, w_{t-3})$$

In summary, the equations for a neural language model with a window size of 3, given one-hot input vectors for each input context word, are:

$$
e = [Ex_{t-3}, Ex_{t-2}, Ex_{t-1}] \\
h = \sigma(We + b) \\
z = Uh \\
\hat{y} = \text{softmax}(z)
$$

<div style="display:flex;align-items:center;justify-content:center;">
<img src="images/neural_language_model.png"  style="width:800px;object-fit:cover;"/>
</div>

## Training the neural language model

The high-level intuition of training neural language models, whether the simple feedforward language models we describe here or the more powerful transformer language models, is the idea of **self-training** or **self-supervision** for learning word representations. In self-training for language modeling, we take a corpus of text as training material and at each time step $t$ ask the model to predict the next word. At first it will do poorly at this task, but since in each case we know the correct answer (it’s the next word in the corpus!) we can easily train it to be better at predicting the correct next word. We call such a model self-supervised because we don’t have to add any special gold labels to the data; the natural sequence of words is its own supervision! We simply train the model to minimize the error in predicting the true next word in the training sequence.

In practice, training the model means setting the parameters $\theta = E,W,U,b$. For some tasks, it’s ok to freeze the embedding layer $E$ with initial _word2vec_ values. Freezing means we use _word2vec_ or some other pretraining algorithm to compute the initial embedding matrix $E$, and then hold it constant while we only modify $W$, $U$, and $b$, i.e., we don’t update $E$ during language model training. However, often we’d like to learn the embeddings simultaneously with training the network. This is useful when the task the network is designed for (like sentiment classification, translation, or parsing) places strong constraints on what makes a good representation for words.

Let’s see how to train the entire model including $E$, i.e. to set all the parameters $\theta = E,W,U,b$. We’ll do this via gradient descent, using error backpropagation on the computation graph to compute the gradient. Training thus not only sets the weights $W$ and $U$ of the network, but also as we’re predicting upcoming words, we’re learning the embeddings $E$ for each word that best predict upcoming words.

The figure below shows the set up for a window size of $N=3$ context words. The input $x$ consists of 3 one-hot vectors, fully connected to the embedding layer via 3 instantiations of the embedding matrix $E$. We don’t want to learn separate weight matrices for mapping each of the 3 previous words to the projection layer. We want one single embedding dictionary $E$ that’s shared among these three. That’s because over time, many different words will appear as $w_{t−2}$ or $w_{t−1}$, and we’d like to just represent each word with one vector, whichever context position it appears in. Recall that the embedding weight matrix $E$ has a column for each word, each a column vector of $d$ dimensions, and hence has dimensionality $d \times |V|$.

<div style="display:flex;align-items:center;justify-content:center;">
<img src="images/training_language_model.png"  style="width:800px;object-fit:cover;"/>
</div>

Generally training proceeds by taking as input a very long text, concatenating all the sentences, starting with random weights, and then iteratively moving through the text predicting each word $w_t$. At each word $w_t$, we use the cross-entropy (negative log likelihood) loss. Recall that the general form for this is:

$$
L_{CE}(\hat{y}, y) = -\log(\hat{y}_i) \quad (\text{where } i \text{ is the correct class})
$$

For language modeling, the classes are the words in the vocabulary, so $\hat{y}_i$ here means the probability that the model assigns to the correct next word $w_t$:

$$
L_{CE} = -\log(p(w_t|w_{t-1}, \cdots ,w_{t-n+1}))
$$

The parameter update for stochastic gradient descent for this loss from step $s$ to $s+1$ is then:

$$
\theta^{s+1} = \theta^s - \eta \frac{\partial[-\log(p(w_t|w_{t-1}, \cdots ,w_{t-n+1}))]}{\partial \theta}
$$

This gradient can be computed in any standard neural network framework which will then backpropagate through $\theta = E,W,U,b$.

Training the parameters to minimize loss will result both in an algorithm for language modeling (a word predictor) but also a new set of embeddings $E$ that can be used as word representations for other tasks.